#**Análise de Sentimentos e Classificação de Textos**

#Objetivos:
    • Identificar diversas aplicações práticas da Análise de Sentimentos em diferentes domínios.
    • Compreender o funcionamento básico do algoritmo Naive Bayes para classificação de textos e análise de sentimentos.
    • Compreender o funcionamento básico do algoritmo de Árvores de Decisão para classificação de textos.
    • Identificar diferentes abordagens para realizar análise de sentimentos.

#**Exemplos do roteiro**

##Exemplo 1 - Análise de sentimentos com a biblioteca NLTK

- **Primeira etapa – Importação das bibliotecas e corpus para realizar a analise**

In [1]:
# Importando as bibliotecas
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Certifique-se de baixar os recursos necessários do NLTK
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

- **Segunda etapa – Inicialização do objeto que realiza a analise e o texto a ser analisado**

In [2]:
# Inicializando o analisador de sentimentos
sid = SentimentIntensityAnalyzer()

# Definindo os textos de exemplo para treinar o modelo
textos = [
    "Este filme é incrível! Eu adorei cada minuto.",
    "Odiei este filme. Foi uma perda de tempo.",
    "O filme foi ok, nada de especial.",
    "Estou muito feliz com o produto, funciona perfeitamente!",
    "Que serviço horrível! Nunca mais compro nessa loja."
]

- **Terceira etapa – Analisando os sentimentos dos textos fornecidos**

In [3]:
# Analisando os sentimentos dos textos
for texto in textos:
    print(f"Texto: {texto}")
    ss = sid.polarity_scores(texto)
    print(f"Pontoação: {ss}")
    if ss['compound'] >= 0.05:
        print("Sentimento: Positivo")
    elif ss['compound'] <= -0.05:
        print("Sentimento: Negativo")
    else:
        print("Sentimento: Neutro")
    print("\n")

Texto: Este filme é incrível! Eu adorei cada minuto.
Pontoação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Odiei este filme. Foi uma perda de tempo.
Pontoação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: O filme foi ok, nada de especial.
Pontoação: {'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'compound': 0.296}
Sentimento: Positivo


Texto: Estou muito feliz com o produto, funciona perfeitamente!
Pontoação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Que serviço horrível! Nunca mais compro nessa loja.
Pontoação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro




##Exemplo 2 - Análise de sentimentos de Avaliações de Filmes com Naives Bayes

- **Primeira etapa: Criação de funções que calculam a quantidade de palavras, e as probabilidades**

In [4]:
import numpy as np

def contar_palavras(avaliacoes, sentimento):
    """Conta a frequência de cada palavra em avaliações de um dado sentimento."""
    contagem = {}
    for avaliacao, sent in avaliacoes:
        if sent == sentimento:
            for palavra in avaliacao.lower().split():
                if palavra in contagem:
                    contagem[palavra] += 1
                else:
                    contagem[palavra] = 1
    return contagem

def calcular_probabilidade_sentimento(avaliacoes, sentimento):
    """Calcula a probabilidade de uma avaliação ser de um dado sentimento."""
    total_avaliacoes = len(avaliacoes)
    contagem_sentimento = sum(1 for _, sent in avaliacoes if sent == sentimento)
    return contagem_sentimento / total_avaliacoes

def calcular_probabilidade_palavra_dado_sentimento(palavra, contagem_palavras, total_palavras_sentimento, total_palavras_vocabulario, alpha=1):
    """Calcula a probabilidade de uma palavra aparecer dado um sentimento (com suavização Laplace)."""
    contagem_palavra = contagem_palavras.get(palavra, 0)
    return (contagem_palavra + alpha) / (total_palavras_sentimento + alpha * total_palavras_vocabulario)


- **Segunda etapa: Criação da função que calcula a classificação da palavra pertencer a um sentimento**

In [5]:
def classificar_avaliacao(avaliacao, avaliacoes):
    """Classifica o sentimento de uma nova avaliação usando Naive Bayes."""

    sentimentos = ["Positivo", "Negativo"]
    probabilidades_sentimento = {}

    # Calcular probabilidades a priori dos sentimentos
    probabilidades_sentimento["Positivo"] = calcular_probabilidade_sentimento(avaliacoes, "Positivo")
    probabilidades_sentimento["Negativo"] = calcular_probabilidade_sentimento(avaliacoes, "Negativo")

    # Contar palavras em cada sentimento
    contagem_palavras_positivo = contar_palavras(avaliacoes, "Positivo")
    contagem_palavras_negativo = contar_palavras(avaliacoes, "Negativo")

    # Calcular total de palavras em cada sentimento
    total_palavras_positivo = sum(contagem_palavras_positivo.values())
    total_palavras_negativo = sum(contagem_palavras_negativo.values())

    # Calcular tamanho do vocabulário
    vocabulario = set()
    for avaliacao, _ in avaliacoes:
        for palavra in avaliacao.lower().split():
            vocabulario.add(palavra)
    total_palavras_vocabulario = len(vocabulario)

    # Calcular probabilidades para cada sentimento
    for sentimento in sentimentos:
        probabilidade_total = probabilidades_sentimento[sentimento]
        contagem_palavras_sentimento = contagem_palavras_positivo if sentimento == "Positivo" else contagem_palavras_negativo
        total_palavras_sentimento = total_palavras_positivo if sentimento == "Positivo" else total_palavras_negativo

        for palavra in avaliacao.lower().split():
            probabilidade_palavra = calcular_probabilidade_palavra_dado_sentimento(
                palavra, contagem_palavras_sentimento, total_palavras_sentimento, total_palavras_vocabulario
            )
            probabilidade_total *= probabilidade_palavra
        probabilidades_sentimento[sentimento] = probabilidade_total

    # Classificar com base na maior probabilidade
    melhor_sentimento = max(probabilidades_sentimento, key=probabilidades_sentimento.get)
    return melhor_sentimento


- **Terceira Etapa – Fornecimento de um modelo para treinamento**

In [7]:
avaliacoes = [
  ("Filme incrível! Adorei a atuação.", "Positivo"),
  ("Roteiro fraco, não recomendo.", "Negativo"),
  ("Atuação boa, mas filme longo.", "Positivo"),
  ("Péssimo! Perdi meu tempo.", "Negativo"),
]

- **Quarta etapa  - Fornecendo um frase para ser avaliada**

In [9]:
# Nova avaliação a ser classificada
nova_avaliacao = input("O que você achou do filme 101 Dalmatas? ")

# Classificar a nova avaliação
sentimento = classificar_avaliacao(nova_avaliacao, avaliacoes)
print(f"A avaliação '{nova_avaliacao}' foi classificada como: {sentimento}")

O que você achou do filme 101 Dalmatas? ok
A avaliação 'ok' foi classificada como: Negativo


- **Quinta etapa – Treine com um novo modelo de dados**

In [10]:
avaliacoes = [
   ("Filme incrível! Adorei a atuação, roteiro e direção.", "Positivo"),
   ("Roteiro fraco, atuação ruim, não recomendo. Perda de tempo.", "Negativo"),
   ("Atuação boa, mas filme um pouco longo e arrastado.", "Positivo"),
   ("Péssimo! Perdi meu tempo, dinheiro e paciência. Horrível.", "Negativo"),
   ("Amei! Elenco maravilhoso, fotografia impecável. Recomendo muito!", "Positivo"),
   ("Chato, previsível e sem graça. Saí no meio do filme.", "Negativo"),
   ("Uma obra-prima! Roteiro inteligente, atuações memoráveis.", "Positivo"),
   ("Cansativo, atuações forçadas, final decepcionante.", "Negativo"),
   ("Divertido e emocionante! Ótimo para assistir com a família.", "Positivo"),
   ("Clichê, sem originalidade, diálogos fracos. Não vale a pena.", "Negativo"),
   ("Surpreendente e envolvente! Me prendeu do início ao fim.", "Positivo"),
   ("Confuso, mal dirigido, edição amadora. Um desastre.", "Negativo"),
   ("Atuação excelente, história cativante, trilha sonora perfeita.", "Positivo"),
   ("Atuações medianas, história arrastada, trilha sonora irritante.", "Negativo"),
   ("Roteiro bem construído, diálogos afiados, direção impecável.", "Positivo"),
   ("Roteiro fraco, diálogos bobos, direção perdida.", "Negativo"),
   ("Uma experiência cinematográfica única! Recomendo a todos os amantes do cinema.", "Positivo"),
   ("Um filme esquecível, sem emoção e sem impacto.", "Negativo"),
   ("Comovente e inspirador! Me fez refletir sobre a vida.", "Positivo"),
   ("Decepcionante e frustrante! Não recomendo nem para passar o tempo.", "Negativo"),
   ("Ótimo, gostei muito",'Positivo')
]

In [11]:
# Testando novamente após novo treinamento
nova_avaliacao = input("O que você achou do filme 101 Dalmatas? ")

# Classificar a nova avaliação
sentimento = classificar_avaliacao(nova_avaliacao, avaliacoes)
print(f"A avaliação '{nova_avaliacao}' foi classificada como: {sentimento}")

O que você achou do filme 101 Dalmatas? ok
A avaliação 'ok' foi classificada como: Positivo
